### Test methods that work with accounts

In [1]:
from dexter.DB import DB, Account, Entry, Transaction, Document, Tag

import re

In [2]:
DB.init()

In [3]:
DB.open('dev')

### Match Account Names

In [4]:
Account.objects(name__contains='checking')

[<Account: checking assets>]

In [5]:
Account.objects(name__contains='food')

[<Account: expenses:food expenses>, <Account: groceries expenses>, <Account: restaurant expenses>]

In [6]:
for a in DB.find_account('food'):
    print(a)

expenses:food expenses
groceries expenses
restaurant expenses


In [7]:
DB.account_name_parts()

{'alice',
 'assets',
 'bank',
 'bob',
 'car',
 'charity',
 'chase',
 'checking',
 'donation',
 'equity',
 'expenses',
 'food',
 'fuel',
 'gift',
 'groceries',
 'home',
 'household',
 'income',
 'interest',
 'liabilities',
 'medical',
 'mortgage',
 'payment',
 'personal',
 'restaurant',
 'savings',
 'ssa',
 'travel',
 'utility',
 'visa',
 'yard',
 'yoyodyne'}

In [8]:
DB.account_name_parts('expenses')

{'alice',
 'bob',
 'car',
 'charity',
 'donation',
 'expenses',
 'food',
 'fuel',
 'gift',
 'groceries',
 'home',
 'household',
 'medical',
 'mortgage',
 'payment',
 'personal',
 'restaurant',
 'travel',
 'utility',
 'yard'}

In [9]:
dct = DB.account_names()

In [10]:
dct

{'equity': ['equity'],
 'assets': ['assets:bank:checking', 'assets:bank:savings'],
 'bank': ['assets:bank:checking', 'assets:bank:savings'],
 'checking': ['assets:bank:checking'],
 'savings': ['assets:bank:savings'],
 'expenses': ['expenses:car',
  'expenses:car:payment',
  'expenses:car:fuel',
  'expenses:donation',
  'expenses:donation:gift',
  'expenses:donation:charity',
  'expenses:food',
  'expenses:food:groceries',
  'expenses:food:restaurant',
  'expenses:home',
  'expenses:home:mortgage',
  'expenses:home:household',
  'expenses:home:utility',
  'expenses:home:yard',
  'expenses:medical',
  'expenses:personal:alice',
  'expenses:personal:bob',
  'expenses:personal:bob',
  'expenses:travel'],
 'car': ['expenses:car', 'expenses:car:payment', 'expenses:car:fuel'],
 'payment': ['expenses:car:payment'],
 'fuel': ['expenses:car:fuel'],
 'donation': ['expenses:donation',
  'expenses:donation:gift',
  'expenses:donation:charity'],
 'gift': ['expenses:donation:gift'],
 'charity': ['exp

In [11]:
dct['expenses']

['expenses:car',
 'expenses:car:payment',
 'expenses:car:fuel',
 'expenses:donation',
 'expenses:donation:gift',
 'expenses:donation:charity',
 'expenses:food',
 'expenses:food:groceries',
 'expenses:food:restaurant',
 'expenses:home',
 'expenses:home:mortgage',
 'expenses:home:household',
 'expenses:home:utility',
 'expenses:home:yard',
 'expenses:medical',
 'expenses:personal:alice',
 'expenses:personal:bob',
 'expenses:personal:bob',
 'expenses:travel']

In [12]:
len(dct['expenses'])

19

In [13]:
dct = DB.account_names('expenses')

In [14]:
dct

{'expenses': ['expenses:car',
  'expenses:car:payment',
  'expenses:car:fuel',
  'expenses:donation',
  'expenses:donation:gift',
  'expenses:donation:charity',
  'expenses:food',
  'expenses:food:groceries',
  'expenses:food:restaurant',
  'expenses:home',
  'expenses:home:mortgage',
  'expenses:home:household',
  'expenses:home:utility',
  'expenses:home:yard',
  'expenses:medical',
  'expenses:personal:alice',
  'expenses:personal:bob',
  'expenses:personal:bob',
  'expenses:travel'],
 'car': ['expenses:car', 'expenses:car:payment', 'expenses:car:fuel'],
 'payment': ['expenses:car:payment'],
 'fuel': ['expenses:car:fuel'],
 'donation': ['expenses:donation',
  'expenses:donation:gift',
  'expenses:donation:charity'],
 'gift': ['expenses:donation:gift'],
 'charity': ['expenses:donation:charity'],
 'food': ['expenses:food',
  'expenses:food:groceries',
  'expenses:food:restaurant'],
 'groceries': ['expenses:food:groceries'],
 'restaurant': ['expenses:food:restaurant'],
 'home': ['expen

In [15]:
len(dct)

20

In [16]:
dct['car']

['expenses:car', 'expenses:car:payment', 'expenses:car:fuel']

In [17]:
DB.account_groups()

['equity',
 'assets:bank:checking',
 'assets:bank:savings',
 'expenses:car',
 'expenses:car:payment',
 'expenses:car:fuel',
 'expenses:donation',
 'expenses:donation:gift',
 'expenses:donation:charity',
 'expenses:food',
 'expenses:food:groceries',
 'expenses:food:restaurant',
 'expenses:home',
 'expenses:home:mortgage',
 'expenses:home:household',
 'expenses:home:utility',
 'expenses:home:yard',
 'expenses:medical',
 'expenses:personal:alice',
 'expenses:personal:bob',
 'expenses:personal:bob',
 'expenses:travel',
 'income:yoyodyne',
 'income:ssa',
 'income:interest',
 'liabilities:chase:visa']

In [18]:
DB.account_groups(['expenses'])

['expenses:car',
 'expenses:car:payment',
 'expenses:car:fuel',
 'expenses:donation',
 'expenses:donation:gift',
 'expenses:donation:charity',
 'expenses:food',
 'expenses:food:groceries',
 'expenses:food:restaurant',
 'expenses:home',
 'expenses:home:mortgage',
 'expenses:home:household',
 'expenses:home:utility',
 'expenses:home:yard',
 'expenses:medical',
 'expenses:personal:alice',
 'expenses:personal:bob',
 'expenses:personal:bob',
 'expenses:travel']

In [19]:
DB.account_groups(['expenses:1'])

['expenses.*']

In [20]:
DB.account_groups(['expenses:2'])

['expenses:car.*',
 'expenses:donation.*',
 'expenses:food.*',
 'expenses:home.*',
 'expenses:medical.*',
 'expenses:travel.*']

In [21]:
DB.account_groups(['expenses:3'])

['expenses:car',
 'expenses:car:payment.*',
 'expenses:car:fuel.*',
 'expenses:donation',
 'expenses:donation:gift.*',
 'expenses:donation:charity.*',
 'expenses:food',
 'expenses:food:groceries.*',
 'expenses:food:restaurant.*',
 'expenses:home',
 'expenses:home:mortgage.*',
 'expenses:home:household.*',
 'expenses:home:utility.*',
 'expenses:home:yard.*',
 'expenses:medical',
 'expenses:personal:alice.*',
 'expenses:personal:bob.*',
 'expenses:personal:bob.*',
 'expenses:travel']

In [22]:
DB.account_groups(['expenses:food'])

['expenses:food', 'expenses:food:groceries', 'expenses:food:restaurant']

In [23]:
DB.account_groups(['expenses:food:2'])

['expenses:food.*']

In [24]:
DB.account_groups(['expenses:food:3'])

['expenses:food', 'expenses:food:groceries.*', 'expenses:food:restaurant.*']

In [25]:
DB.account_groups(['expenses','income'])

['expenses:car',
 'expenses:car:payment',
 'expenses:car:fuel',
 'expenses:donation',
 'expenses:donation:gift',
 'expenses:donation:charity',
 'expenses:food',
 'expenses:food:groceries',
 'expenses:food:restaurant',
 'expenses:home',
 'expenses:home:mortgage',
 'expenses:home:household',
 'expenses:home:utility',
 'expenses:home:yard',
 'expenses:medical',
 'expenses:personal:alice',
 'expenses:personal:bob',
 'expenses:personal:bob',
 'expenses:travel',
 'income:yoyodyne',
 'income:ssa',
 'income:interest']

In [26]:
DB.account_groups(['expenses:1','income:1'])

['expenses.*', 'income.*']

In [27]:
DB.account_groups(['expenses:2','income:2'])

['expenses:car.*',
 'expenses:donation.*',
 'expenses:food.*',
 'expenses:home.*',
 'expenses:medical.*',
 'expenses:travel.*',
 'income:yoyodyne.*',
 'income:ssa.*',
 'income:interest.*']

In [28]:
Account.objects(name__exact='expenses:food:groceries')

[<Account: groceries expenses>]

In [29]:
Account.objects(abbrev__exact='groceries')

[<Account: groceries expenses>]

In [30]:
s = 'expenses:food:groceries'

In [31]:
if acct := Account.objects(name__exact=s) or Account.objects(abbrev__exact=s):
    print(acct[0].name)

expenses:food:groceries


In [32]:
DB.fullname('expenses:food:groceries')

'expenses:food:groceries'

In [33]:
DB.fullname('groceries')

'expenses:food:groceries'

In [34]:
DB.fullname('groc')

In [35]:
Account.objects.get(name='assets:bank:checking')

<Account: checking assets>

In [36]:
Account.objects.get(abbrev='checking')

<Account: checking assets>

Bummer -- would be nice if this returned None instead of raising an exception:

In [37]:
# Account.objects.get(abbrev='foo')

In [38]:
Account.DoesNotExist

dexter.DB.DoesNotExist

In [39]:
Account.objects.first()

<Account: equity equity>

In [40]:
def abbrev(a):
    try:
        acct = Account.objects.get(name=a)
        res = acct.abbrev
    except Account.DoesNotExist:
        res = a
    return res

In [41]:
abbrev('assets:bank:checking')

'checking'

In [42]:
abbrev('assets:bank:whatever')

'assets:bank:whatever'

In [43]:
abbrev('expenses:personal:bob')

MultipleObjectsReturned: 2 or more items returned, instead of 1

### Balances

In [ ]:
DB.select(Entry, account='expenses:food*')

[<Entry: <En 2024-01-02 expenses:food -$500.0 [<Tag.B: '#budget'>]>>, <Entry: <En 2024-02-02 expenses:food -$500.0 [<Tag.B: '#budget'>]>>, <Entry: <En 2024-01-07 expenses:food:groceries +$75.0 []>>, <Entry: <En 2024-01-21 expenses:food:groceries +$175.0 []>>, <Entry: <En 2024-02-07 expenses:food:groceries +$75.0 []>>, <Entry: <En 2024-02-21 expenses:food:groceries +$75.0 []>>]

In [ ]:
for e in DB.select(Entry, account='expenses:food*'):
    print(e)

<En 2024-01-02 expenses:food -$500.0 [<Tag.B: '#budget'>]>
<En 2024-02-02 expenses:food -$500.0 [<Tag.B: '#budget'>]>
<En 2024-01-07 expenses:food:groceries +$75.0 []>
<En 2024-01-21 expenses:food:groceries +$175.0 []>
<En 2024-02-07 expenses:food:groceries +$75.0 []>
<En 2024-02-21 expenses:food:groceries +$75.0 []>


In [ ]:
for t in DB.select(Transaction, tag='#budget'):
    print(t)

<Tr 2024-01-02 expenses:food/expenses:home/expenses:car/expenses:travel/equity -> income:yoyodyne $5000.0 fill envelopes ['#budget']>
<Tr 2024-02-02 expenses:food/expenses:home/expenses:car/expenses:travel/equity -> income:yoyodyne $5000.0 fill envelopes ['#budget']>


In [ ]:
DB.balance('food')

-600.0

In [ ]:
DB.balance('food', budgets=False)

400.0

In [ ]:
DB.balance('food', ending='2024-01-31')

-250.0

In [ ]:
DB.balance('food', ending='2024-01-31', budgets=False)

250.0